First we import all necessary libraries

In [15]:
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
import chart_studio.plotly as py
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import Dash, Input, Output, callback, dash_table
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc

# Data cleaning
As a first step the data is read in as a csv file.

In [16]:
data = pd.read_csv("Example_data_masterthesis.csv", sep = ";")

In [17]:
data.head()

,ID,Tested_system,Software_test,Test_result,Execution_date,Test runtime
0,1,System A,Log-in,1,03/02/2022,00:01:34
1,2,System A,Vertrag_a,1,03/02/2022,00:05:41
2,3,System A,Vertrag_b,1,03/02/2022,00:09:48
3,4,System A,Vertrag_c,1,03/02/2022,00:03:55
4,5,System A,Vertrag_d,1,03/02/2022,00:02:02


As we want to be able to add various different data, we change the column names, so that we can assure they stay the same. The old name of the columns will vary from data to data and will need to be adapted. 

In [18]:
data = data.rename(columns={"Tested_system":"system", "Software_test":"testname", "Test_result":"result", "Execution_date":"date", "Test runtime":"runtime"})

After the data is read in, we want to check, if all data types are set correct. As this is not the case, we set the correct types

In [19]:
data.dtypes

ID           int64
system      object
testname    object
result       int64
date        object
runtime     object
dtype: object

In [20]:
data = data.astype({"result": "bool", "date":"datetime64"})
#data = data.astype({"date":"datetime64"})
data["runtime"] = pd.to_datetime(data["runtime"], format= "%H:%M:%S").dt.time
data["date"] = pd.to_datetime(data["date"], format= "%D:%M:%Y").dt.date
data.head()

,ID,system,testname,result,date,runtime
0,1,System A,Log-in,True,2022-03-02,00:01:34
1,2,System A,Vertrag_a,True,2022-03-02,00:05:41
2,3,System A,Vertrag_b,True,2022-03-02,00:09:48
3,4,System A,Vertrag_c,True,2022-03-02,00:03:55
4,5,System A,Vertrag_d,True,2022-03-02,00:02:02


Now we check if there are any null values

In [21]:
data.isna().any()

ID          False
system      False
testname    False
result      False
date        False
runtime     False
dtype: bool

Here we set a column as an id.

In [22]:
data['id'] = data['ID']
data.set_index('id', inplace=True, drop=False)
print(data.columns)

Index(['ID', 'system', 'testname', 'result', 'date', 'runtime', 'id'], dtype='object')


In [23]:
dataset=data.groupby('result').size().reset_index(name='count')
dataset.replace(to_replace= "1", value = "Passed")
dataset
#fig = px.bar(dataset, x='result', y='count') 
#fig.show()

,result,count
0,False,5
1,True,15


In [24]:
#data[data['system'] == "System B"]
option = []
for i in (data["system"].unique()):
    option.append(i)
print(option)

['System A', 'System B']


# Dashboard creation
Next we start building the dashboard.

In [25]:
app = Dash(__name__)

In [26]:
option = []
        
for i in (data["system"].unique()):
    option.append(i)

app.layout = html.Div([

   html.H1("Software test results", style={'text-align': 'center', 'padding-left': '5%'}),

    # Data table
    html.Div([
        dash_table.DataTable(
            id='datatable',
                    columns=[
                        {"name": i, "id": i, "deletable": False, "selectable": True, "hideable": True}
                        # only a few columns in our data table are hideable
                        if i == "ID" or i == "system" or i == "runtime" or i == "id"
                        else {"name": i, "id": i, "deletable": False, "selectable": True}
                        for i in data.columns
                    ],

                data=data.to_dict('records'),
                filter_action="native",
                sort_action="native",
                sort_mode="single",
                column_selectable = "multi", 
                fixed_rows={'headers': True},
                row_selectable="multi",     
                row_deletable=True,        
                selected_columns=[],       
                selected_rows=[],           
                page_action="native",
                fill_width=True,       
                style_table = { 'overflowY': 'auto','height': '900px'},
                style_cell={                
                'minWidth': 120, 'maxWidth': 120, 'width': 120
                  },
                style_cell_conditional=[    
                    {
                        'if': {'column_id': c},
                        'textAlign': 'left'
                    } for c in ['testname', 'system']
                ],
                style_data={                
                    'whiteSpace': 'normal',
                    'height': 'auto'
                },
            ),
        ], style={"width": "50%", 'padding-top': '1%', 'display': 'float', 'float' : 'left'}),

   # Dropdown menue
    html.Div(children=[
        dcc.Dropdown(id='dropdown_1',
                options=[
                         {'label': option[0], 'value': 'system'},
                         {'label': option[1], 'value': 'testname'},
                         #{'label': option[2], 'value': 'testname'},
                         #{'label': option[3], 'value': 'testname'},
                ],
                #value='system',
                multi=True,
                clearable=True,
                #style={'display': 'inline-block'}
            ),
        dcc.Graph(id='linechart'), #style={'display': 'inline-block', 'margin-left': '15px', 'verticalAlign': 'middle'}
        dcc.Graph(id='piechart') #style={'display': 'inline-block', 'padding-left': '11%', 'padding-top': '4%'}

    ],style={"width": "45%", 'display': 'float', 'float' : 'right', 'padding-top': '1%','margin-left': 'auto', 'margin-right': '0'}),
    
    html.Br()
], style={"padding": "5px"})
 

In [27]:
@app.callback(
    [Output('piechart', 'figure'),
     Output('linechart', 'figure')],
    [Input('datatable', 'selected_rows'),
     Input('dropdown_1', 'value')]
)
def update_data(dropdown_1, all_rows_data):

    dff = pd.DataFrame(all_rows_data)
    dff_sub=dff.groupby('result').size().reset_index(name='count')

    pie_chart=px.pie(
            data_frame=dff_sub,
            names='result',
            values='count',
            hole=.3,
           # labels={'countriesAndTerritories':'Countries'}
            )

    line_chart = px.line(
            data_frame=dff,
            x='date',
            y='runtime',
            color='system',
            #labels={'countriesAndTerritories':'Countries', 'dateRep':'date'},
            )
    line_chart.update_layout(uirevision='foo')

    return (pie_chart,line_chart)



#@app.callback( 
#    [#Output(component_id='piechart', component_property='figure'),
#    Output(component_id='linechart', component_property='figure')],
#    [Input(component_id='dropdown_1', component_property="value"),
#    Input(component_id='datatable', component_property="selected_rows")]
#)
#def update_data(option_slct, all_rows_data, slctd_row_indices, slct_rows_names, slctd_rows,
 #              order_of_rows_indices, order_of_rows_names, actv_cell, slctd_cell): #slctd_rows, dropdown_1

   # dff = pd.DataFrame(all_rows_data)
   # dff_sub=dff.groupby('result').size().reset_index(name='count')

    # Pie chart definition
   # pie_chart=px.pie(
   #     data_frame=dff_sub,
   #     values= 'count',
   #     names= 'result',
   # ),
    #Line chart definition
   # line_chart = px.line(
   #         data_frame=dff,
   #         x='date',
   #         y='runtime',
   #         color= option_slct,
            #labels={'countriesAndTerritories':'Countries', 'dateRep':'date'},
   #         )
    #line_chart.update_layout(transition_duration=500)

#    return line_chart      


In [28]:
if __name__ == '__main__':
    app.run_server(debug=False)
    

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Mar/2022 22:27:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\gaert\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispatch
    response.set_d

" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\gaert\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispatch
    response.set_d

127.0.0.1 - - [28/Mar/2022 22:27:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Mar/2022 22:27:51] "POST /_dash-update-component HTTP/1.1" 500 -
